In [27]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [28]:
data=pd.read_csv(r'C:\Users\USER\final covid malaria.csv')

In [29]:
data

,Time (hours),Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Symptom 6,Symptom 7,Symptom 8,Symptom 9,Symptom 10,Symptom 11,Symptom 12,Symptom 13,Diagnosis
0,48.0,FEVER,SHIVERING,GENERAL FEELING OF DISCOMFORT,HEADACHE,NAUSEA,VOMITING,DIARRHEA,ABDOMINAL PAIN,MUSCLE OR JOINT PAIN,FATIGUE,RAPID BREATHING,RAPID HEART RATE,COUGH,MALARIA
1,48.0,FEVER,HEADACHE,CHILLS,SWEATING,FATIGUE,NAUSEA,VOMTING,BODY ACHES,GENERALLY FEELING SICK,,,,,MALARIA
2,24.0,FEVER,CHILLS,SWEATS,HEADACHE,NAUSEA,VOMITING,BODY PAIN,GENERAL DISCOMFORT,,,,,,MALARIA
3,12.0,FEVER,CHILLS,NAUSEA,VOMITING,SWEATING,HEADACHES,WEAKNESS,A GENERAL FEELING OF BEING UNWELL,,,,,,MALARIA
4,48.0,POOR APPETITE,CHILLS,FEVER,HIGH TEMPERATURE,SWEATING,NAUSEA,HEADACHE,BODY PAIN,,,,,,MALARIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,264.0,DIFFICULTY BREATHING,COUGH,VOMITING,RUNNY NOSE,SORE THROAT,DIARRHEA,BODY ACHES,FATIGUE,HEADACHE,NEW LOSS OF TASTE,FEVER,NaN,NaN,COVID
196,48.0,FEVER,COUGH,DIARRHEA,FATIGUE,NEW LOSS OF TASTE,BODY ACHES,DIFFICULTY BREATHING,SORE THROAT,VOMITING,HEADACHE,RUNNY NOSE,NaN,NaN,COVID
197,312.0,SORE THROAT,VOMITING,FEVER,COUGH,BODY ACHES,DIARRHEA,HEADACHE,RUNNY NOSE,DIFFICULTY BREATHING,NEW LOSS OF TASTE,FATIGUE,NaN,NaN,COVID
198,168.0,FEVER,RUNNY NOSE,DIFFICULTY BREATHING,FATIGUE,SORE THROAT,DIARRHEA,COUGH,HEADACHE,BODY ACHES,NEW LOSS OF TASTE,VOMITING,NaN,NaN,COVID


In [30]:
data=data.replace(np.nan,' ')

In [31]:
data.columns

Index(['Time (hours)', 'Symptom 1', 'Symptom 2', 'Symptom 3', 'Symptom 4',
       'Symptom 5', 'Symptom 6', 'Symptom 7', 'Symptom 8', 'Symptom 9',
       'Symptom 10', 'Symptom 11', 'Symptom 12', 'Symptom 13', 'Diagnosis'],
      dtype='object')

In [32]:
data['Symptoms']= data['Symptom 1']+','+data['Symptom 2']+','+data['Symptom 3']+','+data['Symptom 4']+','+data['Symptom 5']+','+data['Symptom 6']+','+data['Symptom 7']+','+data['Symptom 8']+','+data['Symptom 9']+','+data['Symptom 10']+','+data['Symptom 11']+','+data['Symptom 12']+','+data['Symptom 13']

In [33]:
data=data.drop(['Symptom 1', 'Symptom 2', 'Symptom 3', 'Symptom 4',
       'Symptom 5', 'Symptom 6', 'Symptom 7', 'Symptom 8', 'Symptom 9',
       'Symptom 10', 'Symptom 11', 'Symptom 12', 'Symptom 13'],axis=1)

In [34]:
data.columns

Index(['Time (hours)', 'Diagnosis', 'Symptoms'], dtype='object')

In [38]:
data=data.reindex(columns=['Time (hours)', 'Symptoms','Diagnosis'])

In [39]:
data

,Time (hours),Symptoms,Diagnosis
0,48.0,"FEVER,SHIVERING,GENERAL FEELING OF DISCOMFORT,...",MALARIA
1,48.0,"FEVER,HEADACHE,CHILLS,SWEATING,FATIGUE,NAUSEA,...",MALARIA
2,24.0,"FEVER,CHILLS,SWEATS,HEADACHE,NAUSEA,VOMITING,B...",MALARIA
3,12.0,"FEVER,CHILLS,NAUSEA,VOMITING,SWEATING,HEADACHE...",MALARIA
4,48.0,"POOR APPETITE,CHILLS,FEVER,HIGH TEMPERATURE,SW...",MALARIA
...,...,...,...
195,264.0,"DIFFICULTY BREATHING,COUGH,VOMITING,RUNNY NOSE...",COVID
196,48.0,"FEVER,COUGH,DIARRHEA,FATIGUE,NEW LOSS OF TASTE...",COVID
197,312.0,"SORE THROAT,VOMITING,FEVER,COUGH,BODY ACHES,DI...",COVID
198,168.0,"FEVER,RUNNY NOSE,DIFFICULTY BREATHING,FATIGUE,...",COVID


In [86]:
x=data['Symptoms']
y=data["Diagnosis"]

In [87]:
import string
punc=string.punctuation
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)

In [88]:
import spacy
nlp = spacy.load('en_core_web_sm')

def text_cleaner(sentence):
    doc=nlp(sentence)
    
    tokens=[]
    for token in doc:
        if token.lemma_!="-PRON-":
            temp=token.lemma_.lower().strip()
        else:
            temp=token.lower_
        tokens.append(temp)
        
    cleaned_tokens=[]
    for token in tokens:
        if token not in stopwords and token not in punc:
            cleaned_tokens.append(token)
    return cleaned_tokens
    
        

In [89]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [90]:
tfidf=TfidfVectorizer(tokenizer=text_cleaner)

In [91]:
classifier=LinearSVC()

In [92]:
from sklearn.model_selection import train_test_split

In [93]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

In [94]:
x_train.shape,y_train.shape

((140,), (140,))

In [95]:
clf=Pipeline([('tfidf',tfidf),('clf',classifier)])

In [96]:
clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_cleaner at 0x000002724265AD40>)),
                ('clf', LinearSVC())])

In [97]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [98]:
y_pred=clf.predict(x_test)

In [99]:
confusion_matrix(y_test,y_pred)

array([[30,  0],
       [ 0, 30]], dtype=int64)

In [100]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       COVID       1.00      1.00      1.00        30
     MALARIA       1.00      1.00      1.00        30

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [125]:
val=['cough,difficulty breathing']

In [126]:
pred=clf.predict(val)

In [127]:
pred

array(['COVID'], dtype=object)